In [1]:
"""
Lab Work #4
"""
from time import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from os import getcwd
import re

trainData = pd.read_csv(getcwd() + "\\LWs\\LW_4\\data\\train.csv", sep=",")
testData = pd.read_csv(getcwd() + "\\LWs\\LW_4\\data\\test.csv", sep=",")

In [3]:
trainData.describe()

,SalaryNormalized
count,60000.000000
mean,34084.931500
std,17593.539646
min,5000.000000
25%,21500.000000
50%,30000.000000
75%,42500.000000
max,200000.000000


In [5]:
trainData.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [6]:
trainData['FullDescription'] = trainData['FullDescription'].apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', x.lower()))
testData['FullDescription'] = testData['FullDescription'].apply(lambda x: re.sub('[^a-zA-Z0-9]', ' ', x.lower()))

trainData['LocationNormalized'].fillna('nan', inplace=True)
testData['LocationNormalized'].fillna('nan', inplace=True)

trainData['ContractTime'].fillna('nan', inplace=True)
testData['ContractTime'].fillna('nan', inplace=True)


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

"""
TF-IDF (от англ. TF — term frequency, IDF — inverse document frequency)
 — статистическая мера, используемая для оценки важности слова 
в контексте документа, являющегося частью коллекции документов 
или корпуса. Вес некоторого слова пропорционален количеству 
употребления этого слова в документе, и обратно пропорционален 
частоте употребления слова в других документах коллекции.
"""

vectorizer = TfidfVectorizer(input='content', encoding='utf-8', 
decode_error='strict', strip_accents=None, lowercase=True, 
preprocessor=None, tokenizer=None, analyzer='word', 
stop_words=None, min_df=5)

vectorizer2 = TfidfVectorizer(input='content', encoding='utf-8', 
decode_error='strict', strip_accents=None, lowercase=True, 
preprocessor=None, tokenizer=None, analyzer='word', 
stop_words=None)


X_train = vectorizer.fit_transform(trainData['FullDescription'])
X_test = vectorizer2.fit_transform(testData['FullDescription'])

In [17]:
from sklearn.feature_extraction import DictVectorizer

enc = DictVectorizer()
X_train_categ = enc.fit_transform(trainData[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.fit_transform(testData[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [18]:
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

clf = Ridge(alpha=1.0, fit_intercept=False, solver='lsqr')
clf.fit(hstack([X_train, X_train_categ]), trainData['SalaryNormalized'])

Ridge(alpha=1.0, copy_X=True, fit_intercept=False, max_iter=None,
   normalize=False, random_state=None, solver='lsqr', tol=0.001)

In [34]:
test = hstack([X_test, X_test_categ])
prediction = clf.predict(test)
print(prediction)



ValueError: dimension mismatch